<a href="https://colab.research.google.com/github/Vivek084c/image_processing/blob/main/fcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Jun 19 19:16:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
def load():
  #dataset preprocess


In [27]:

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
import scipy.io
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger


In [28]:
label_map={"0": "Background", "1": "Hat", "2": "Hair", "3": "Sunglasses", "4": "Upper-clothes", "5": "Skirt", "6": "Pants", "7": "Dress", "8": "Belt", "9": "Left-shoe", "10": "Right-shoe", "11": "Face", "12": "Left-leg", "13": "Right-leg", "14": "Left-arm", "15": "Right-arm", "16": "Bag", "17": "Scarf"}
rgb_color_map=[[0,0,0], [0,0,128], [0,0,255], [0,85,0], [51,0,170], [0,85,255], [85,0,0], [221,119,0], [0,85,85], [85,85,0], [0,51,85], [128,86,52], [0,28,0], [255,0,0], [221,170,51], [225,225,0], [170,255,85], [85,255,170]]
classes=["Background",  "Hat",  "Hair",  "Sunglasses",  "Upper-clothes", "Skirt", "Pants",  "Dress", "Belt",  "Left-shoe", "Right-shoe", "Face", "Left-leg", "Right-leg", "Left-arm", "Right-arm", "Bag", "Scarf"]

COLOR_MAP=[k for k in range(18)]

In [29]:
def create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [30]:
#creating the directories
create_path("files")
create_path("train/input")
create_path("train/catogories")




In [31]:
x_path="/content/drive/MyDrive/dataset/train_1/img_0.jpg"
y_path="/content/drive/MyDrive/dataset/Categories_1_rgb"

model_path=os.path.join("files","model.h5")
csv_path=os.path.join("files","data.csv")

In [7]:
import shutil

In [8]:

for i in range(300):
  print(i)
  shutil.copy(f"/content/drive/MyDrive/dataset/train_1/img_{i}.jpg", "/content/train/input")

for i in range(300):
  print(i)
  shutil.copy(f"/content/drive/MyDrive/dataset/Categories_1_rgb/img_{i}.png", "/content/train/catogories")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [32]:
def load_dataset(path):
  train_x=sorted(glob(os.path.join(path, "train", "input", "*")))
  train_y=sorted(glob(os.path.join(path, "train","catogories", "*")))


  #splitting the dataset
  train_x,valid_x,test_x=train_x[:180],train_x[180:240],train_x[240:]
  train_y,valid_y,test_y=train_y[:180],train_y[180:240],train_y[240:]



  return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [10]:
print(len(glob(os.path.join("/content", "train", "input", "*"))))
print(len(glob(os.path.join("/content", "train","catogories", "*"))))

300
300


In [11]:
(train_x, train_y), (valid_x, valid_y), (test_x, test_y)=load_dataset("/content")
print("____")
print(train_x[:5])
print(train_y[:5])
print("____")
print(f"len of train_x: {len(train_x)}, len of train_y: {len(train_y)}" )
print(f"len of test_x: {len(test_x)}, len of test_y: {len(test_y)}" )
print(f"len of tvalid_x: {len(valid_x)}, len of tvalid_y: {len(valid_y)}" )


____
['/content/train/input/img_0.jpg', '/content/train/input/img_1.jpg', '/content/train/input/img_10.jpg', '/content/train/input/img_100.jpg', '/content/train/input/img_101.jpg']
['/content/train/catogories/img_0.png', '/content/train/catogories/img_1.png', '/content/train/catogories/img_10.png', '/content/train/catogories/img_100.png', '/content/train/catogories/img_101.png']
____
len of train_x: 180, len of train_y: 180
len of test_x: 60, len of test_y: 60
len of tvalid_x: 60, len of tvalid_y: 60


In [33]:
colormap=[[0,0,0], [0,0,128], [0,0,255], [0,85,0], [51,0,170], [0,85,255], [85,0,0], [221,119,0], [0,85,85], [85,85,0], [0,51,85], [128,86,52], [0,28,0], [255,0,0], [221,170,51], [225,225,0], [170,255,85], [85,255,170]]
classes=["Background",  "Hat",  "Hair",  "Sunglasses",  "Upper-clothes", "Skirt", "Pants",  "Dress", "Belt",  "Left-shoe", "Right-shoe", "Face", "Left-leg", "Right-leg", "Left-arm", "Right-arm", "Bag", "Scarf"]

In [13]:
def read_image(x):
  x=cv2.imread(x, cv2.IMREAD_COLOR)
  x=cv2.resize(x, (320, 320))
  x=x/255.0
  x=x.astype(np.float32)
  return x


In [14]:
def read_mask(x):
  #reading the image and resizing it
  x=cv2.imread(x, cv2.IMREAD_COLOR)
  x=cv2.resize(x, (320, 320))
  # processing the mask and generating the output vectore of the model
  output=[]
  for color in colormap:
    cmap=np.all(np.equal(x, color) ,axis=-1)
    output.append(cmap)
  output = np.stack(output)
  output = output.astype(np.uint8)
  return output

In [15]:
a=train_x[0]
b=train_y[0]

In [18]:
print(a)
img = read_image(a)
print(b)
mask = read_mask(b)

print(img.shape)
print(mask.shape)

/content/train/input/img_0.jpg
/content/train/catogories/img_0.png
(320, 320, 3)
(18, 320, 320)


In [34]:
def read_image_mask(x , y):
  #reading the files
  x=cv2.imread(x, cv2.IMREAD_COLOR)
  y=cv2.imread(y, cv2.IMREAD_COLOR)
  if x is None or y is None:
      raise ValueError("Error reading image or mask. Check the file paths.")

  #resizing
  x=cv2.resize(x, (320, 320))
  y=cv2.resize(y, (320, 320))

  #image processing
  x=x/255.0
  x=x.astype(np.float32)

  #mask processing

  #processing the first five class only
  c=0
  output=[]
  for color in colormap:
    cmap=np.all(np.equal(y, color) ,axis=-1)
    output.append(cmap)
    c+=1
    if c==5:
      break
  output = np.stack(output, axis=-1)
  output = output.astype(np.uint8)


  return x,output

alpha,beta = read_image_mask(train_x[0],train_y[0])
print(f"the shape of alpha is : {alpha.shape}")
print(f"the shape of beta is : {beta.shape}")


def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()
        image, mask = read_image_mask(x, y)
        return image, mask

    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.uint8])
    image.set_shape([320, 320, 3])
    mask.set_shape([320, 320, 5])

    return image, mask

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset




the shape of alpha is : (320, 320, 3)
the shape of beta is : (320, 320, 5)


In [ ]:
def preprocess(x, y):
  # x and y represent the image and the mask path
  def f(x,y):
    x=x.decode()
    y=y.decode()

    x=read_image(x)
    y=read_mask(y)

    return x,y
  image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.uint8])

  image.set_shape([320, 320, 3])
  mask.set_shape([320, 320, 5])

  return image, mask

In [ ]:
def tf_dataset(x, y, batch=8):
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  dataset = dataset.shuffle(buffer_size=5000)
  dataset = dataset.map(preprocess)
  dataset = dataset.batch(batch)
  dataset = dataset.prefetch(2)
  return dataset

In [35]:
import tensorflow as tf
#head sub model

def fcn8(input_shape=(320, 320, 3), num_classes=5):
    inputs = tf.keras.layers.Input(shape=input_shape)

    # Block 1
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 2
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 3
    x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 4
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 5
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Fully Convolutional Layers (FC Layers)
    x = tf.keras.layers.Conv2D(4096, (7, 7), activation='relu', padding='same')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Conv2D(4096, (1, 1), activation='relu', padding='same')(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    # Classification layer (output layer)
    x = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax', padding='same')(x)

    # Upsampling for FCN-8
    x = tf.keras.layers.Conv2DTranspose(num_classes, kernel_size=(4, 4), strides=(2, 2), padding='same')(x)
    x = tf.keras.layers.Conv2DTranspose(num_classes, kernel_size=(4, 4), strides=(2, 2), padding='same')(x)
    x = tf.keras.layers.Conv2DTranspose(num_classes, kernel_size=(16, 16), strides=(8, 8), padding='same')(x)


    model = tf.keras.Model(inputs=inputs, outputs=x)
    return model


In [ ]:
def data_generator(a,b):
    while True:
        # Generate or load data
        X, y = a,b # Ensure load_data() returns valid X and y
        if X or y is None:
          if X is None and y is None  :
            return "both none"
          if X is None:
            return "x is None"
          else:
            return "y is None"

        return "valid"




In [ ]:
# print(data_generator(train_dataset,valid_dataset ))
print(valid_dataset.batch)

<bound method DatasetV2.batch of <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 320, 320, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 320, 320, 5), dtype=tf.uint8, name=None))>>


In [44]:
"""
HYPER-PARAMETERS
"""
batch_size=8
lr=1e-8
num_epoch=30

"""
classes and colormap
"""
colormap=[[0,0,0], [0,0,128], [0,0,255], [0,85,0], [51,0,170], [0,85,255], [85,0,0], [221,119,0], [0,85,85], [85,85,0], [0,51,85], [128,86,52], [0,28,0], [255,0,0], [221,170,51], [225,225,0], [170,255,85], [85,255,170]]
classes=["Background",  "Hat",  "Hair",  "Sunglasses",  "Upper-clothes", "Skirt", "Pants",  "Dress", "Belt",  "Left-shoe", "Right-shoe", "Face", "Left-leg", "Right-leg", "Left-arm", "Right-arm", "Bag", "Scarf"]


"""
loading the dataset
"""
(train_x, train_y), (valid_x, valid_y), (test_x, test_y)=load_dataset("/content")

"""
dataset - pipeline
"""

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

"""
MODEL
"""
model = fcn8(input_shape=(320, 320, 3), num_classes=5)
model.compile(
    loss = "categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(lr)
)
# model.summary()

"""
Training
"""
callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path, append=True),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)

]





In [45]:
model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=num_epoch,
        callbacks=callbacks
    )



Epoch 1/30
23/23 [==============================] - ETA: 0s - loss: 5.3693
Epoch 1: val_loss improved from inf to 5.37040, saving model to files/model.h5
23/23 [==============================] - 78s 1s/step - loss: 5.3693 - val_loss: 5.3704 - lr: 1.0000e-08
Epoch 2/30
23/23 [==============================] - ETA: 0s - loss: 5.3690
Epoch 2: val_loss improved from 5.37040 to 5.37012, saving model to files/model.h5
23/23 [==============================] - 37s 2s/step - loss: 5.3690 - val_loss: 5.3701 - lr: 1.0000e-08
Epoch 3/30
23/23 [==============================] - ETA: 0s - loss: 5.3688
Epoch 3: val_loss improved from 5.37012 to 5.37002, saving model to files/model.h5


KeyboardInterrupt: 

In [ ]:
# Example usage:
model = fcn8(input_shape=(320, 320, 3), num_classes=5)
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 320, 320, 3)]     0         
                                                                 
 conv2d_16 (Conv2D)          (None, 320, 320, 64)      1792      
                                                                 
 conv2d_17 (Conv2D)          (None, 320, 320, 64)      36928     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 160, 160, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 160, 160, 128)     73856     
                                                                 
 conv2d_19 (Conv2D)          (None, 160, 160, 128)     147584    
                                                           

In [ ]:
import tensorflow as tf

#body sub model

def fcn8(input_shape=(640, 640, 3), num_classes=21):
    inputs = tf.keras.layers.Input(shape=input_shape)

    # Block 1
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 2
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 3
    x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 4
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 5
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Fully Convolutional Layers (FC Layers)
    x = tf.keras.layers.Conv2D(4096, (7, 7), activation='relu', padding='same')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Conv2D(4096, (1, 1), activation='relu', padding='same')(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    # Classification layer (output layer)
    x = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax', padding='same')(x)

    # Upsampling for FCN-8
    x = tf.keras.layers.Conv2DTranspose(num_classes, kernel_size=(4, 4), strides=(2, 2), padding='same')(x)
    x = tf.keras.layers.Conv2DTranspose(num_classes, kernel_size=(4, 4), strides=(2, 2), padding='same')(x)
    x = tf.keras.layers.Conv2DTranspose(num_classes, kernel_size=(16, 16), strides=(8, 8), padding='same')(x)

    model = tf.keras.Model(inputs=inputs, outputs=x)
    return model

# Example usage:
model = fcn8(input_shape=(640, 384, 3), num_classes=21)
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 640, 384, 3)]     0         
                                                                 
 conv2d_16 (Conv2D)          (None, 640, 384, 64)      1792      
                                                                 
 conv2d_17 (Conv2D)          (None, 640, 384, 64)      36928     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 320, 192, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 320, 192, 128)     73856     
                                                                 
 conv2d_19 (Conv2D)          (None, 320, 192, 128)     147584    
                                                           

In [ ]:
import tensorflow as tf

# Define your FCN-8 model (assuming you've already defined `fcn8` function as provided earlier)
model = fcn8(input_shape=(640, 384, 3), num_classes=21)

# Define your loss function (e.g., categorical crossentropy for multi-class segmentation)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# Define learning rate schedule with decay
initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,  # adjust decay steps based on your preference
    decay_rate=0.96,
    staircase=True)

# Define optimizer with momentum and learning rate schedule
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Prepare dummy data (replace with your actual data loading and preprocessing)
import numpy as np
x_train = np.random.randn(100, 640, 384, 3)  # example random input data
y_train = np.random.randint(0, 21, size=(100, 640, 384))  # example random labels

# Dummy training loop (replace with your actual training loop)
batch_size = 10
epochs = 10

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}:")
    for i in range(0, len(x_train), batch_size):
        x_batch = x_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]

        # Forward pass
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss_value = loss_fn(y_batch, logits)

        # Backward pass
        gradients = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # Print batch loss (optional)
        print(f"  Batch {i//batch_size+1}/{len(x_train)//batch_size}, Loss: {loss_value.numpy():.4f}")

# Evaluate the model (optional)
# Replace x_test and y_test with your test data
# loss, accuracy = model.evaluate(x_test, y_test)
# print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


Epoch 1/10:


  Batch 1/10, Loss: 7.1355
  Batch 2/10, Loss: 3.0847
  Batch 3/10, Loss: 3.0735
  Batch 4/10, Loss: 3.0700
  Batch 5/10, Loss: 3.0687
  Batch 6/10, Loss: 3.0676
  Batch 7/10, Loss: 3.0669
  Batch 8/10, Loss: 3.0663
  Batch 9/10, Loss: 3.0658
  Batch 10/10, Loss: 3.0650
Epoch 2/10:
  Batch 1/10, Loss: 3.0648
  Batch 2/10, Loss: 3.0646
  Batch 3/10, Loss: 3.0643
  Batch 4/10, Loss: 3.0635
  Batch 5/10, Loss: 3.0634
  Batch 6/10, Loss: 3.0631
  Batch 7/10, Loss: 3.0628
  Batch 8/10, Loss: 3.0626
  Batch 9/10, Loss: 3.0624
  Batch 10/10, Loss: 3.0619
Epoch 3/10:
  Batch 1/10, Loss: 3.0618
  Batch 2/10, Loss: 3.0617
  Batch 3/10, Loss: 3.0615
  Batch 4/10, Loss: 3.0609
  Batch 5/10, Loss: 3.0609
  Batch 6/10, Loss: 3.0607
  Batch 7/10, Loss: 3.0605
  Batch 8/10, Loss: 3.0604
  Batch 9/10, Loss: 3.0602
  Batch 10/10, Loss: 3.0598
Epoch 4/10:
  Batch 1/10, Loss: 3.0597
  Batch 2/10, Loss: 3.0597
  Batch 3/10, Loss: 3.0596
  Batch 4/10, Loss: 3.0590
  Batch 5/10, Loss: 3.0590
  Batch 6/10, Lo

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9, decay=0.0005)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

In [ ]:
print(len(os.listdir("drive/MyDrive/dataset/train/images")))
print(len(os.listdir("drive/MyDrive/dataset/train/labels")))
print(len(os.listdir("drive/MyDrive/dataset/val/images")))
print(len(os.listdir("drive/MyDrive/dataset/val/labels")))

350
350
114
114


In [ ]:
# path: /content/drive/MyDrive
# train: dataset/train/images
# val: dataset/val/images

# nc: 2
# names: ["head","body"]

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.6/780.6 kB 9.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from ultralytics import YOLO

In [ ]:
model=YOLO("yolov8n.yaml")
model.train(data="data.yaml", epochs=50,batch=-1)

Ultralytics YOLOv8.2.31 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=data.yaml, epochs=50, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, lin

100%|██████████| 755k/755k [00:00<00:00, 38.5MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 195MB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (Tesla T4) 14.75G total, 0.23G reserved, 0.07G allocated, 14.46G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     3011238       8.195         0.315         24.93         111.4        (1, 3, 640, 640)                    list
     3011238       16.39         0.298         20.03         42.81        (2, 3, 640, 640)                    list
     3011238       32.78         0.533         19.58         47.01        (4, 3, 640, 640)                    list
     3011238       65.56         1.063         26.74         51.58        (8, 3, 640, 640)                    list
     3011238       131.1         2.003         42.59         67.49       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 72 for CUDA:0 8.87G/14.75G (60%) ✅


...

                   all        114        228      0.977      0.984      0.994      0.708



50 epochs completed in 0.131 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.31 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.46s/it]


                   all        114        228      0.977      0.984      0.994      0.708
                  head        113        113      0.974      0.976      0.993      0.675
                  body        114        115      0.981      0.991      0.995      0.742
Speed: 0.2ms preprocess, 1.6ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c110055b430>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
! cd /content/runs

In [ ]:
!zip -r dataset.zip runs/

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/train/ (stored 0%)
  adding: runs/detect/train/args.yaml (deflated 53%)
  adding: runs/detect/train/train_batch0.jpg (deflated 6%)
  adding: runs/detect/train/P_curve.png (deflated 19%)
  adding: runs/detect/train/R_curve.png (deflated 14%)
  adding: runs/detect/train/F1_curve.png (deflated 13%)
  adding: runs/detect/train/train_batch1.jpg (deflated 5%)
  adding: runs/detect/train/weights/ (stored 0%)
  adding: runs/detect/train/weights/best.pt (deflated 11%)
  adding: runs/detect/train/weights/last.pt (deflated 11%)
  adding: runs/detect/train/confusion_matrix_normalized.png (deflated 34%)
  adding: runs/detect/train/train_batch202.jpg (deflated 10%)
  adding: runs/detect/train/confusion_matrix.png (deflated 38%)
  adding: runs/detect/train/train_batch2.jpg (deflated 6%)
  adding: runs/detect/train/labels_correlogram.jpg (deflated 47%)
  adding: runs/detect/train/val_batch0_pred.jpg (deflated 6%)
  ad

In [ ]:
from google.colab import files
files.download('dataset.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!nvidia-smi

Wed Jun 12 09:16:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0              30W /  70W |   1251MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--